### **6.14 - Dependencies between your DAGs with the ExternalTaskSensor**

En el vídeo anterior hemos visto cómo podemos disparar un DAG desde otro DAG utilizando el TriggerDagRunOperator.  Si recuerdas, cuando un DAG se dispara desde otro DAG permanecen independientes. La ejecución del DAG controlador no impacta en la ejecución del DAG target.  En este vídeo te voy a mostrar un nuevo operador llamado **`ExternalTaskSensor`** que hace lo contrario que el TriggerDagRunOperator. Como siempre, vamos a empezar con un caso de uso sencillo.  Imaginemos que tenemos dos DAGs donde el primero extrae datos de las bases de datos de producción y luego el segundo agrega los datos para empujar el resultado a otra base de datos.  Una cosa que podemos hacer es esperar a que el primer DAG termine antes de iniciar el segundo DAG. Implementando el ExternalTaskSensor podemos posponer el inicio del segundo DAG hasta que el primero finalice con éxito.  ¿Qué es el ExternalTaskSensor? 

<center><img src="https://i.postimg.cc/XvqdF841/a1068.png"></center>

Volviendo a los vídeos anteriores, hemos aprendido que un sensor es un tipo especial de operador que espera a que algo suceda.  Pues eso es exactamente lo que hace el ExternalTaskSensor pero esta vez espera a que una tarea externa termine antes de pasar a las siguientes tareas, y así es como se pueden crear dependencias entre DAGs.  Déjame mostrarte cómo funciona.  

<center><img src="https://i.postimg.cc/ryxWSz2T/a1069.png"></center>

Primero, necesitamos dos DAGs, DAG 1 y DAG 2. Ambos DAGs tienen el mismo schedule_interval.  Esto es importante de entender. El ExternalTaskSensor asume que ambos DAGs están en la misma execution date.  Si tus DAGs no tienen la misma execution date, pero, aun así, tienen el mismo schedule_interval, aún puedes usar el sensor modificando los parámetros execution_delta y execution_date_fn. Volveré sobre ellos más adelante en el resto del curso.  Por lo tanto, ambos DAGs tienen los mismos schedule intervals y se están ejecutando.  En este ejemplo, el DAG 2 esperará a que termine la última tarea del DAG 1 para pasar a las siguientes.  ¿Cómo? Usando el ExternalTaskSensor como primera tarea esperando a que la tarea t3 del DAG 1 tenga éxito como se muestra aquí.  Así que después de un tiempo, t1 y t2 del DAG 1 tuvieron éxito, ahora t3 se está ejecutando mientras que la tarea del sensor está esperando que t3 tenga éxito también.  

<center><img src="https://i.postimg.cc/vB6n2Z1W/a1070.png"></center>

Cada minuto, el sensor comprueba si el estado de t3 ha cambiado.  En algún momento, t3 termina, el sensor se da cuenta de este nuevo estado y entonces el DAG 2 puede continuar con las siguientes tareas t4 y t5 hasta que el DAGRun termine también.  

<center><img src="https://i.postimg.cc/2yQZm97y/a1071.png"></center>

Así es como puedes hacer que tu DAG dependa de otro en Airflow.  Bien, déjame darte algunas notas importantes sobre el ExternalTaskSensor. 

<center><img src="https://i.postimg.cc/3N5vsQpz/a1072.png"></center>

En primer lugar, debes mantener el mismo schedule interval entre tus DAGs dependientes.  Si no, puedes atascar tu sensor, que es el comportamiento que solemos tener cuando intentamos utilizar tanto el TriggerDagRunOperator como el ExternalTaskSensor. ¿Por qué?  Porque, el DAG target debería tener la misma execution date que el DAG controlador lo cual no es posible ya que el DAG target tiene su schedule_interval configurado a None. En resumen, si intentas activar un DAG target y luego esperas a que termine utilizando el ExternalTaskSensor, incluso después de que el DAGRun del destino haya terminado, el sensor seguirá esperando y eso, indefinidamente.  Ahora bien, si sólo utilizas el sensor y tus DAGs no tienen la misma execution date, puedes solucionarlo definiendo o bien el parámetro execution_delta o bien el parámetro execution_date_fn. Nunca ambos. El parámetro **`execution_delta`** es la diferencia de tiempo con la ejecución anterior a mirar. Por defecto es la misma execution_delta que la tarea actual.  Por ejemplo, si tienes dos DAGs ejecutándose cada hora, pero el DAG 1 se dispara a las 10am y el DAG 2 a las 10:30am, sólo tienes que asignar un objeto timedelta de 30 minutos al parámetro execution_delta.  Al hacerlo, las execution dates serán las mismas para el ExternalTaskSensor. El segundo parámetro **`execution_date_fn`** sigue el mismo propósito de execution_delta pero de una forma más flexible. Este parámetro espera una función que reciba la execution_date actual y devuelva la execution_date deseada para consultar.  Básicamente, dentro de esta función usted puede hacer cualquier procesamiento que desee.  Finalmente, por defecto el sensor usará el método "poke" para comprobar si la tarea del otro DAG tiene éxito o no.  Eso está bien si esperas que el tiempo de ejecución (runtime) del sensor sea corto o si necesitas un intervalo de poke corto.  De lo contrario, debería utilizar el modo "reschedule", que se considera la mejor práctica con el ExternalTaskSensor para evitar malgastar worker slots.  